# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [52]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [42]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,izberbash,42.5696,47.8645,4.52,74,22,0.66,RU,1737407400
1,1,karoi,-16.8099,29.6925,18.26,92,100,1.16,ZW,1737407401
2,2,ushuaia,-54.8000,-68.3000,11.18,81,100,1.46,AR,1737407403
3,3,klyuchi,52.2667,79.1667,-3.61,95,100,7.42,RU,1737407404
4,4,waitangi,-43.9535,-176.5597,14.90,80,95,3.58,NZ,1737407405


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    frame_width=800,
    frame_height=600,
    color="City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [44]:
# Narrow down cities that fit criteria and drop any results with null values
# I filtered for Max Temp between 20 and 30 C, Humidity between 30 and 70, and Cloudiness less that 50%.
ideal_weather_df = city_data_df[(city_data_df["Max Temp"]>=20) &
             (city_data_df['Max Temp']<=30) &
             (city_data_df["Humidity"] >=30) &
             (city_data_df["Humidity"]<=70) &
             (city_data_df["Cloudiness"]<50)]

# Drop any rows with null values
ideal_weather_df.info()  #to check for null values
# N/A all the filtered columns contained data. If there were missing values I would use the following code:
#clean_ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 42 to 487
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     17 non-null     int64  
 1   City        17 non-null     object 
 2   Lat         17 non-null     float64
 3   Lng         17 non-null     float64
 4   Max Temp    17 non-null     float64
 5   Humidity    17 non-null     int64  
 6   Cloudiness  17 non-null     int64  
 7   Wind Speed  17 non-null     float64
 8   Country     17 non-null     object 
 9   Date        17 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 1.5+ KB


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,georgetown,5.4112,100.3354,26.96,65,20,3.09,MY,1737407452
46,46,minas de marcona,-15.2119,-75.1103,20.72,61,21,8.19,PE,1737407457
50,50,puerto baquerizo moreno,-0.9000,-89.6000,28.55,65,40,7.20,EC,1737407465
67,67,hawaiian paradise park,19.5933,-154.9731,25.50,63,20,4.12,US,1737407445
84,84,cabo san lucas,22.8909,-109.9124,23.20,43,0,5.14,MX,1737407247
180,180,pisco,-13.7000,-76.2167,25.03,61,0,7.20,PE,1737407631
190,190,holualoa,19.6228,-155.9522,24.75,64,0,3.09,US,1737407643
195,195,mahina,-17.5065,-149.4890,29.87,66,0,8.23,PF,1737407646
233,233,george town,5.4112,100.3354,26.96,65,20,3.09,MY,1737407416
240,240,ilo,-17.6394,-71.3375,25.48,61,0,3.09,PE,1737407706


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country","Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,georgetown,MY,5.4112,100.3354,65,
46,minas de marcona,PE,-15.2119,-75.1103,61,
50,puerto baquerizo moreno,EC,-0.9000,-89.6000,65,
67,hawaiian paradise park,US,19.5933,-154.9731,63,
84,cabo san lucas,MX,22.8909,-109.9124,43,
180,pisco,PE,-13.7000,-76.2167,61,
190,holualoa,US,19.6228,-155.9522,64,
195,mahina,PF,-17.5065,-149.4890,66,
233,george town,MY,5.4112,100.3354,65,
240,ilo,PE,-17.6394,-71.3375,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"}    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
   
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
georgetown - nearest hotel: Page 63 hostel
minas de marcona - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: Dory's House
hawaiian paradise park - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
pisco - nearest hotel: La Portada
holualoa - nearest hotel: Kona Hotel
mahina - nearest hotel: Motu Martin
george town - nearest hotel: Page 63 hostel
ilo - nearest hotel: Sunset Hotel Ilo
paratinga - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
caluula - nearest hotel: No hotel found
kailua-kona - nearest hotel: PACIFIC 19 Kona
bubaque - nearest hotel: Chez Julio
damara - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth


,City,Country,Lat,Lng,Humidity,Hotel Name
42,georgetown,MY,5.4112,100.3354,65,Page 63 hostel
46,minas de marcona,PE,-15.2119,-75.1103,61,No hotel found
50,puerto baquerizo moreno,EC,-0.9000,-89.6000,65,Dory's House
67,hawaiian paradise park,US,19.5933,-154.9731,63,No hotel found
84,cabo san lucas,MX,22.8909,-109.9124,43,Comfort Rooms
180,pisco,PE,-13.7000,-76.2167,61,La Portada
190,holualoa,US,19.6228,-155.9522,64,Kona Hotel
195,mahina,PF,-17.5065,-149.4890,66,Motu Martin
233,george town,MY,5.4112,100.3354,65,Page 63 hostel
240,ilo,PE,-17.6394,-71.3375,61,Sunset Hotel Ilo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    frame_width = 800,
    frame_height = 600,
    size=50,
    hover_cols=["Country", "Hotel Name", "Humidity"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)